In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import glob
from copy import deepcopy

In [8]:
results_folder = "results"
df_experiment = pd.DataFrame()

In [9]:
print(f"{len(os.listdir(results_folder))} results")

18 results


In [10]:
for experiment_folder in os.listdir(results_folder):
    experiment_files = os.listdir(os.path.join(results_folder,experiment_folder))

    if len(experiment_files) != 3:
        continue
    
    for file in experiment_files:
        
        read_file = os.path.join(results_folder,experiment_folder,file)

        
        
        if "config" in file:
            df_config =pd.read_csv(read_file)
            
        if "test_metrics" in file:
            df_test = pd.read_csv(read_file)
            
        if "benchmark" in file:
            df_benchmark = pd.read_csv(read_file)
            
            avg_time = df_benchmark.Time.mean()
            avg_pop = df_benchmark.Avg.iloc[-1]
            best_pop = df_benchmark.Best.iloc[-1]
            worst_pop = df_benchmark.Worst.iloc[-1]
            
            df_benchmark = pd.DataFrame({"Time": [avg_time],"Avg":[avg_pop],"Worst":[worst_pop],"Best":[best_pop]})

    row = pd.concat((df_test,df_benchmark,df_config),axis=1)

    df_experiment = pd.concat((df_experiment,row))


df_test = deepcopy(df_test)
df_benchmark = deepcopy(df_benchmark)
df_test[:] = np.NaN
df_benchmark[:] = np.NaN
for experiment_folder in os.listdir(results_folder):
    experiment_files = os.listdir(os.path.join(results_folder,experiment_folder))

    if len(experiment_files) < 3:

        
        read_file = os.path.join(results_folder,experiment_folder,"config.csv")
        df_config =pd.read_csv(read_file)

        if df_config.algorithm.item()!="rfs":
            continue

        row = pd.concat((df_test,df_benchmark,df_config),axis=1)
        print(row)
    
        df_experiment = pd.concat((df_experiment,row))

df_experiment = df_experiment.reset_index(drop=True)

In [11]:
df_metrics = df_experiment[df_experiment.backend_prefer=="processes"].reset_index(drop=True)

In [12]:
df_experiment.groupby(["model","algorithm","dataset"])[["F1","Accuracy","ROC AUC"]].mean(1).unstack(-1).unstack(0).stack()

F1                                   Accuracy  \
dataset            gina_agnostic hiva_agnostic sylva_agnostic gina_agnostic   
algorithm model                                                               
ga_joblib logistic      0.781341      0.461538       0.948571      0.783862   
          xgboost       0.932945      0.333333       0.931818      0.933718   
ga_seq    logistic      0.804665      0.461538       0.948571      0.806916   
          xgboost       0.932945      0.333333       0.931818      0.933718   
ga_spark  logistic      0.804665      0.461538       0.948571      0.806916   
          xgboost       0.932945      0.333333       0.931818      0.933718   

                                                      ROC AUC                \
dataset            hiva_agnostic sylva_agnostic gina_agnostic hiva_agnostic   
algorithm model                                                               
ga_joblib logistic      0.966903       0.993750      0.783858      0.693873   
          xgboost       0.971631       0.991667      0.933745      0.600000   
ga_seq    logistic      0.966903       0.993750      0.806918      0.693873   
          xgboost       0.971631       0.991667      0.933745      0.600000   
ga_spark  logistic      0.966903       0.993750      0.806918      0.693873   
          xgboost       0.971631       0.991667      0.933745      0.600000   

                                   
dataset            sylva_agnostic  
algorithm model                    
ga_joblib logistic       0.965182  
          xgboost        0.958824  
ga_seq    logistic       0.965182  
          xgboost        0.958824  
ga_spark  logistic       0.965182  
          xgboost        0.958824

In [ ]:
# # FOLDER REMOVING SCRIPT, SKIP IT #

# import shutil
# ct = 0
# for experiment_folder in os.listdir(results_folder):
#     experiment_files = os.listdir(os.path.join(results_folder,experiment_folder))

#     # if len(experiment_files) != 3:
#     #     continue
    
#     for file in experiment_files:
        
#         read_file = os.path.join(results_folder,experiment_folder,file)

#         if "config" in file:
#             df_config =pd.read_csv(read_file)
        
#             if df_config["algorithm"][0] == "ga_spark":
#                 # print(df_config["algorithm"])
#                 # print(df_config["model"])
#                 # print(df_config["dataset"])
#                 # print(read_file)
#                 print(os.path.join(results_folder,experiment_folder))
#                 # shutil.rmtree(os.path.join(results_folder,experiment_folder))
#                 ct += 1
# print(ct)

In [ ]:
# # TEST SCRIPT, SKIP #

# ct = 0
# algor = "baseline_metrics"
# for experiment_folder in os.listdir(results_folder):
#     experiment_files = os.listdir(os.path.join(results_folder,experiment_folder))

#     # if len(experiment_files) != 3:
#     #     continue
    
#     for file in experiment_files:
        
#         read_file = os.path.join(results_folder,experiment_folder,file)

#         if "config" in file:
#             df_config =pd.read_csv(read_file)
        
#             if df_config["algorithm"][0] == algor:
#                 print()
#                 print(df_config["model"][0])
#                 print(df_config["dataset"][0])
#                 print(os.path.join(results_folder,experiment_folder))
#                 ct += 1
# print(ct)

In [ ]:
models = ["mlp","xgboost","logistic"]
datasets = ["gina_agnostic","hiva_agnostic","sylva_agnostic"]
algorithms = ["baseline_metrics","rfs","ga_seq","random","ga_joblib","ga_spark"][::-1]

In [ ]:
df_metrics = df_experiment.groupby(["dataset","model","algorithm"])[["F1","Accuracy","ROC AUC"]].mean().unstack(level="dataset").reindex(index=algorithms, level=1).reindex(index=models,level=0).stack(0,dropna=False).unstack()#.swaplevel(0, 1).sort_index(axis=1, level=0)
df_metrics = df_metrics.applymap(lambda x: str(np.round(x,3)))
df_metrics.loc["mlp"].loc["rfs"] = "NaN"
df_metrics

In [ ]:
df_metrics.rename(level="algorithm",mapper={algo:algo.replace("_","\_") for algo in algorithms},inplace=True)
df_metrics.rename(level="algorithm",mapper={"baseline\_metrics":"baseline"},inplace=True)
df_metrics.rename(level="dataset",columns={data:data.replace("_","\_") for data in datasets},inplace=True)
print(df_metrics.to_latex(index=True,multicolumn_format="c"))

In [ ]:
df_times = df_experiment[df_experiment.backend_prefer=="processes"].reset_index(drop=True).groupby(["dataset","model","algorithm"])[["Time"]].mean().unstack(level="dataset").reindex(index=algorithms,level=1).reindex(index=models,level=0)
df_times = df_times.applymap(lambda x: str(np.round(x,3)))
df_times.columns = df_times.columns.droplevel(0)
df_times

In [ ]:
df_times.rename(level="algorithm",mapper={algo:algo.replace("_","\_") for algo in algorithms},inplace=True)
df_times.rename(level="algorithm",mapper={"baseline\_metrics":"baseline"},inplace=True)
df_times.rename(level="dataset",columns={data:data.replace("_","\_") for data in datasets},inplace=True)
print(df_times.to_latex(index=True,multicolumn_format="c"))

## Jaccard Overlap

In [ ]:
import itertools

In [ ]:
list(itertools.product(algorithms,algorithms))

In [ ]:
from scipy.spatial.distance import jaccard

In [ ]:
def feature_jaccard_overlap(df,dataset,algorithms):
    pairwise_comparisons = list(itertools.product(algorithms,algorithms))
    df = df[df.dataset==dataset]
    df_overlap = pd.DataFrame(index=algorithms,columns=algorithms,dtype=float)
    for algo1,algo2 in pairwise_comparisons:
        try:
            chromosome1 = eval(df[df.algorithm==algo1]["Best Chromosome"].values[0].replace(" ",","))
            chromosome2 = eval(df[df.algorithm==algo2]["Best Chromosome"].values[0].replace(" ",","))
            overlap = 1-jaccard(chromosome1,chromosome2)
            # print(chromosome1,chromosome2)
            # print(algo1,algo2,overlap)
            df_overlap.loc[algo1.replace("\\",""),algo2.replace("\\","")] = overlap
        except:
            continue

    return df_overlap

df_subset = df_experiment[(df_experiment.backend_prefer=="processes") & (df_experiment.model=="logistic")].reset_index(drop=True)
df_logistic_overlap = feature_jaccard_overlap(df=df_subset,dataset="sylva_agnostic",algorithms=algorithms)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns

# Plotting heatmaps on each subplot
for i, ax in enumerate(axes):
    df_subset = df_experiment[(df_experiment.backend_prefer=="processes") & (df_experiment.model=="logistic")].reset_index(drop=True)
    df_overlap = feature_jaccard_overlap(df=df_subset,dataset="sylva_agnostic",algorithms=algorithms)
    sns.heatmap(df_overlap, ax=ax)
    ax.set_title(f'Heatmap {i+1}')

plt.tight_layout()
plt.show()

In [ ]:
data = {
    (model, dataset): feature_jaccard_overlap(df=df_experiment[(df_experiment.model==model)],dataset="sylva_agnostic",algorithms=algorithms)  # Sample data for each combination
    for model in models
    for dataset in datasets
}

# Creating a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(15, 15))  # 3 rows, 3 columns

# Plotting heatmaps on each subplot
for i, model in enumerate(models):
    for j, dataset in enumerate(datasets):
        sns.heatmap(data[(model, dataset)], ax=axes[i, j], cmap='coolwarm', vmin=0, vmax=1, annot=True, fmt='.2f')
        axes[i, j].set_title(f'{model} - {dataset}')
        axes[i, j].set_xlabel('')  # Remove x-axis label for better readability
        axes[i, j].set_ylabel('') 

plt.suptitle("Jaccard Overlap In Feature Selection")
plt.tight_layout()
plt.savefig(os.path.join("images","jaccard_overlap.pdf"),dpi=1600)
plt.show()

## EXAMPLE IMAGE

In [ ]:
from colorama import Fore, Back, Style
from IPython.display import display, Latex

In [ ]:
from sklearn.datasets import load_iris
X,y = load_iris(return_X_y=True,as_frame=True)
print(Fore.RED + "Full Dataset")
print(Fore.BLACK + f"{X[:5].to_string(index=False)}")
print()

c_l = np.array([0,0,1,1])
print(Fore.RED + "\nChromosome l")
display(Latex('$c_{l}$='+f'{c_l}'))
print()

X_ga = X.iloc[:,c_l==1]

print(Fore.RED + "Genetic Algorithm Subset")
print(Fore.BLACK + f"{X_ga[:5].to_string(index=False)}")